In [4]:
import json

In [9]:
city_file = 'sal.json'
file_name = 'demo.json'
city_dict = {}

In [3]:
# def whether_process(full_name, city_dict):
#     out = False
#     wrong_list = ['australia', 'new south wales', 'queensland', 'victoria', 'western australia', 'south australia',
#                   'tasmania',
#                   '	australian capital territory', 'northern territory']
#     if full_name not in wrong_list:
#         if full_name in city_dict.keys():
#             out = True
#     return out

In [5]:
# import ijson
# # Load city data
# with open(city_file, 'r', encoding='utf-8') as sal_file:
#     for prefix, event, value in ijson.parse(sal_file):
#         if prefix.endswith('.gcc'):
#             location = prefix.split('.')[0]
#             if value[1] == 'g' or value == '8acte':
#                 city_dict[location] = value

In [6]:
from dateutil import parser


def split_time_by_period(timestamp):
    # 将时间戳字符串转换为datetime对象
    dt = parser.parse(timestamp)

    # 获取小时部分
    hour = dt.hour

    # 根据小时划分时间段
    if 3 < hour <= 11:
        period = "morning"
    elif 11 <= hour < 19:
        period = "afternoon"
    else:
        period = "evening"

    return period


In [7]:
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request


# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

# Tasks:
# emoji, emotion, hate, irony, offensive, sentiment
# stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary

task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

# download label mapping
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)

def sentiment_m1(text):
    text = preprocess(text)
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    return labels[ranking[0]]

In [ ]:
# 读取 JSON 文件并提取字段写入新的 JSON 文件
with open(file_name, 'r') as f_in, open("final.json", "w") as outfile:
    # 跳过第一行
    f_in.readline()
    # 逐行读取并解析 JSON
    counter = 1
    for line in f_in:
        # 解析 JSON
        original_data = json.loads(line)
        # 获取时间戳
        timestamp = original_data["created_at"]

       # 计算时间段
        period = split_time_by_period(timestamp)
        sentiment = sentiment_m1(original_data["text"])

        new_data = {
        "id": original_data["id"],
        "GCC": original_data["GCC"],
        'full_name' : original_data["includes"]["places"][0]["full_name"],
        "bbox": original_data["includes"]["places"][0]["geo"]["bbox"][:2],
        "text": original_data["text"],
        "tokens": original_data["value"]["tokens"],
        "author_id": original_data["author_id"],
        "context_annotations": original_data["context_annotations"],
        "created_at": original_data["created_at"],
        "retweet_count": original_data["public_metrics"]["retweet_count"],
        "reply_count": original_data["public_metrics"]["reply_count"],
        "like_count": original_data["public_metrics"]["like_count"],
        "quote_count": original_data["public_metrics"]["quote_count"],

        "lang": original_data["lang"],
        "time_period": period,

        'sentiment' : sentiment
        }
        counter += 1
        if counter % 100000 == 0:
            print(counter)
        outfile.write(json.dumps(new_data, ensure_ascii=False))
        outfile.write('\n')


1690000
1700000
1710000
1720000
1730000
1740000
1750000
1760000
1770000
1780000
1790000
1800000
All data have been saved!


1684134
